# Finite volume methods for the Darcy equation

In this tutorial we present how to solve a Darcy equation with [PorePy](https://github.com/pmgbergen/porepy) by using two finite volume methods: two-point flux approximation (TPFA) and multi-point flux approximation (MPFA).

## Exercise 3

We consider the primal formulation of the Darcy problem: the only unknown is the pressure $p$.

Let $\Omega$ be the domain of interest with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$k$ the matrix permeability, we want to solve the following problem: find $p$ such that
$$
\nabla \cdot (- k\nabla p) = 0
\quad \text{in } \Omega
$$
with boundary conditions:
$$
\left\{
\begin{array}{ll}
p = 10^6 & \text{ on } \partial_{top} \Omega \\
p = 0 & \text{ on } \partial_{bottom} \Omega\\
-\nabla p \cdot \nu = 0  & \text{ on } \partial_{left} \Omega \cup \partial_{right} \Omega
\end{array}
\right.
$$
$\Omega$ and the permeability $k$ are taken from the spe10 benchmark case, see [10.2118/66599-MS](http://dx.doi.org/10.2118/66599-MS), by selecting one or multiple layers. In the latter case on the two additional boundary we impose null flux.

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

In [7]:
import os

cwd_folder = os.getcwd()
spe10_folder = cwd_folder + "/spe10/"

import sys

sys.path.insert(1, spe10_folder)

import numpy as np
import porepy as pp
import pygeon as pg

from spe10 import Spe10

We first set some data that will be used in the sequel

In [8]:
key = "flow"
selected_layers = 35

Let's start first to import the data related to the spe10, grid and permeability

In [9]:
# Define the class with the corresponding layer(s)
spe10 = Spe10(selected_layers)
# For simplicity we extract the grid form the class spe10
sd = spe10.sd

# Read the permeability associated to the given layer(s)
perm_folder = spe10_folder + "/perm/"
spe10.read_perm(perm_folder)
perm_dict = spe10.perm_as_dict()

We set now the data, in particular the permeability tensor by specifying all its diagonal components (the off-diagonal are assumed zero) and the boundary conditions.

In [10]:
# Permeability
perm = pp.SecondOrderTensor(
    kxx=perm_dict["kxx"], kyy=perm_dict["kyy"], kzz=perm_dict["kzz"]
)

# Boundary conditions
b_faces = sd.tags["domain_boundary_faces"].nonzero()[0]
b_face_centers = sd.face_centers[:, b_faces]

# define outflow and inflow type boundary conditions, left and right boundary
out_flow = np.isclose(b_face_centers[1, :], spe10.full_physdims[1])
in_flow = np.isclose(b_face_centers[1, :], 0)

# define the labels and values for the boundary faces
labels = np.array(["neu"] * b_faces.size)
labels[np.logical_or(in_flow, out_flow)] = "dir"

bc_val = np.zeros(sd.num_faces)
bc_val[b_faces[out_flow]] = 1e6

bc = pp.BoundaryCondition(sd, b_faces, labels)

# Collect all parameters in a dictionary
parameters = {"second_order_tensor": perm, "bc": bc, "bc_values": bc_val}
data = pp.initialize_default_data(sd, {}, key, parameters)

For simplicity, we consider the TPFA scheme to solve the problem

In [11]:
tpfa = pp.Tpfa(key)
tpfa.discretize(sd, data)
A_tpfa, b_tpfa = tpfa.assemble_matrix_rhs(sd, data)

ls = pg.LinearSystem(A_tpfa, b_tpfa)
cell_p = ls.solve()

Reconstruct the flux at each face of the grid

In [12]:
mat_discr = data[pp.DISCRETIZATION_MATRICES][key]

q = mat_discr["flux"] @ cell_p + mat_discr["bound_flux"] @ bc_val

We post process the pressure to compute the flux for each face and the projected in each cell for visualization purposes.

In [13]:
# to export the flux we consider the virtual version of the rt0 to work on any cell type
vrt0 = pg.VRT0(key)

# compute an operator for evaluating the flux in the cell centers
proj = vrt0.eval_at_cell_centers(sd)

# construct the P0 flux reconstruction, no reshape is needed
cell_q = proj @ q

Finally, we export the computed solutions and the permeability field.

In [14]:
save = pp.Exporter(sd, "sol", folder_name="ex3")

data_to_export = [
    ("kxx", np.log10(perm_dict["kxx"])),
    ("kyy", np.log10(perm_dict["kyy"])),
    ("kzz", np.log10(perm_dict["kzz"])),
    ("cell_p", cell_p),
    ("cell_q", cell_q),
]
save.write_vtu(data_to_export)

In [15]:
# Consistency check
assert np.isclose(np.linalg.norm(cell_p), 67086884.14163634)
assert np.isclose(np.linalg.norm(cell_q), 203723.40664094014)